In [5]:
import os
from dotenv import load_dotenv
from load_azure_ml import get_azure_ml_client

load_dotenv()

datastores_uri = os.getenv("DATASTORES_URI")

ml_client = get_azure_ml_client()

cluster_name = "sams-16core-DS5-v2"
print(ml_client.compute.get(cluster_name))

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


enable_node_public_ip: true
id: /subscriptions/56539498-d3d8-4a3b-92f4-f3b098a11d1e/resourceGroups/continuous_review_ms_and_ucl/providers/Microsoft.MachineLearningServices/workspaces/EPPI_DEV/computes/sams-16core-DS5-v2
idle_time_before_scale_down: 120
location: westeurope
max_instances: 1
min_instances: 0
name: sams-16core-DS5-v2
provisioning_state: Succeeded
size: Standard_DS5_v2
ssh_public_access_enabled: false
tier: dedicated
type: amlcompute



In [8]:
from azure.ai.ml import Input, load_component
from azure.ai.ml.dsl import pipeline

# find_single_model = ml_client.components.get(
#     "find_single_model_for_classifier_workbench"
# )

find_single_model = load_component(
    source="./final_components/find_single_model/find_single_model.yml"
)

input_data = ml_client.data.get(name="debunking_review_data", version="1.0.0")

hparam_search_ranges = Input(
    path="./user_inputs/uri_folders/hyperparameter_search_ranges/", type="uri_folder"
)


@pipeline(default_compute=cluster_name, name="find_single_model_pipeline")
def find_single_model_pipeline(
    labelled_data,
    unlabelled_data,
    hparam_search_ranges,
    title_header,
    abstract_header,
    label_header,
    positive_class_value,
    model_name,
    max_n_search_iterations,
    nfolds,
    num_cv_repeats,
    timeout,
    use_early_terminator,
    max_stagnation_iterations,
    wilcoxon_trial_pruner_threshold,
    use_worse_than_first_two_pruner,
    shap_num_display,
):
    find_run = find_single_model(
        labelled_data=labelled_data,
        unlabelled_data=unlabelled_data,
        title_header=title_header,
        abstract_header=abstract_header,
        label_header=label_header,
        positive_class_value=positive_class_value,
        model_name=model_name,
        hparam_search_ranges=hparam_search_ranges,
        max_n_search_iterations=max_n_search_iterations,
        nfolds=nfolds,
        num_cv_repeats=num_cv_repeats,
        timeout=timeout,
        use_early_terminator=use_early_terminator,
        max_stagnation_iterations=max_stagnation_iterations,
        wilcoxon_trial_pruner_threshold=wilcoxon_trial_pruner_threshold,
        use_worse_than_first_two_pruner=use_worse_than_first_two_pruner,
        shap_num_display=shap_num_display,
    )

    return find_run.outputs


first_pipeline = find_single_model_pipeline(
    labelled_data=input_data,
    unlabelled_data=input_data,
    hparam_search_ranges=hparam_search_ranges,
    title_header="title",
    abstract_header="abstract",
    label_header="included",
    positive_class_value=1,
    model_name="lightgbm",
    max_n_search_iterations=100,
    nfolds=3,
    num_cv_repeats=1,
    timeout=None,
    use_early_terminator=False,
    max_stagnation_iterations=None,
    wilcoxon_trial_pruner_threshold=None,
    use_worse_than_first_two_pruner=False,
    shap_num_display=20,
)

import datetime

t = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
print(t)

first_pipeline.outputs.unlabelled_tfidf_scores.path = f"{datastores_uri}/EPPI_DEV/datastores/workspaceblobstore/paths/azureml/outputs/{t}/tfidf_scores"
first_pipeline.outputs.labelled_tfidf_scores.path = f"{datastores_uri}/EPPI_DEV/datastores/workspaceblobstore/paths/azureml/outputs/{t}/tfidf_scores"
first_pipeline.outputs.labels.path = f"{datastores_uri}/EPPI_DEV/datastores/workspaceblobstore/paths/azureml/outputs/{t}/labels"
first_pipeline.outputs.best_hparams.path = f"{datastores_uri}/EPPI_DEV/datastores/workspaceblobstore/paths/azureml/outputs/{t}/best_hparams"
first_pipeline.outputs.trained_model.path = f"{datastores_uri}/EPPI_DEV/datastores/workspaceblobstore/paths/azureml/outputs/{t}/trained_model"


first_pipeline_job = ml_client.jobs.create_or_update(
    first_pipeline, experiment_name="find_model_sdk"
)


print(f"current time: {datetime.datetime.now()}")

2025-03-13-14-50-07


pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute o

current time: 2025-03-13 14:50:13.595654


In [9]:
# Wait until the job completes
ml_client.jobs.stream(first_pipeline_job.name)

RunId: hungry_plate_p1l59x9pbq
Web View: https://ml.azure.com/runs/hungry_plate_p1l59x9pbq?wsid=/subscriptions/56539498-d3d8-4a3b-92f4-f3b098a11d1e/resourcegroups/continuous_review_ms_and_ucl/workspaces/eppi_dev

Streaming logs/azureml/executionlogs.txt

[2025-03-13 14:50:15Z] Submitting 1 runs, first five are: d2aebc0e:124f8915-8e09-43c5-9e96-aeda9f24c0a6
[2025-03-13 15:00:50Z] Completing processing run id 124f8915-8e09-43c5-9e96-aeda9f24c0a6.

Execution Summary
RunId: hungry_plate_p1l59x9pbq
Web View: https://ml.azure.com/runs/hungry_plate_p1l59x9pbq?wsid=/subscriptions/56539498-d3d8-4a3b-92f4-f3b098a11d1e/resourcegroups/continuous_review_ms_and_ucl/workspaces/eppi_dev



In [10]:
# Get the completed job details
completed_job = ml_client.jobs.get(first_pipeline_job.name)

# Replace 'your_output_name' with the actual output name from your component definition
output_path = completed_job.outputs["labels"].path
print("Output path:", output_path)


Output path: azureml://subscriptions/56539498-d3d8-4a3b-92f4-f3b098a11d1e/resourcegroups/continuous_review_ms_and_ucl/workspaces/EPPI_DEV/datastores/workspaceblobstore/paths/azureml/outputs/2025-03-13-14-50-07/labels


In [11]:
# output = ml_client.jobs.download(
#     first_pipeline_job.name, download_path="pipeline_outputs/", all=True
# )

In [11]:
# FINISH THE SECOND PIPELINE WHEN BACK
threshold = -8.1

In [12]:
from azure.ai.ml import Input

threshold_model_analysis = load_component(
    source="./final_components/threshold_model_analysis/threshold_model_analysis.yml"
)

labelled_tfidf_scores = Input(
    path=first_pipeline.outputs.labelled_tfidf_scores.path, type="uri_folder"
)
labels = Input(path=first_pipeline.outputs.labels.path, type="uri_folder")
model_params = Input(path=first_pipeline.outputs.best_hparams.path, type="uri_folder")


@pipeline(default_compute=cluster_name, name="threshold_model_analysis_pipeline")
def threshold_model_analysis_pipeline(
    labelled_tfidf_scores,
    labels,
    model_name,
    model_params,
    threshold,
    nfolds,
    histogram_num_cv_repeats,
    confusion_num_cv_repeats,
):
    analysis_run = threshold_model_analysis(
        labelled_tfidf_scores=labelled_tfidf_scores,
        labels=labels,
        model_name=model_name,
        model_params=model_params,
        threshold=threshold,
        nfolds=nfolds,
        histogram_num_cv_repeats=histogram_num_cv_repeats,
        confusion_num_cv_repeats=confusion_num_cv_repeats,
    )

    return analysis_run.outputs


second_pipeline = threshold_model_analysis_pipeline(
    labelled_tfidf_scores=labelled_tfidf_scores,
    labels=labels,
    model_name="lightgbm",
    model_params=model_params,
    threshold=threshold,
    nfolds=3,
    histogram_num_cv_repeats=100,
    confusion_num_cv_repeats=1,
)

second_pipeline_job = ml_client.jobs.create_or_update(
    second_pipeline, experiment_name="find_model_sdk"
)

import datetime

print(f"current time: {datetime.datetime.now()}")

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored


current time: 2025-03-13 15:08:15.042156


In [19]:
ml_client.jobs.stream(second_pipeline_job.name)

RunId: sad_holiday_srm2832r9q
Web View: https://ml.azure.com/runs/sad_holiday_srm2832r9q?wsid=/subscriptions/56539498-d3d8-4a3b-92f4-f3b098a11d1e/resourcegroups/continuous_review_ms_and_ucl/workspaces/eppi_dev

Streaming logs/azureml/executionlogs.txt

[2025-03-12 14:46:29Z] Submitting 1 runs, first five are: 3bcf0cb5:9e5ff7ef-caee-471f-b3cc-8a6874b237d5
[2025-03-12 14:58:40Z] Completing processing run id 9e5ff7ef-caee-471f-b3cc-8a6874b237d5.

Execution Summary
RunId: sad_holiday_srm2832r9q
Web View: https://ml.azure.com/runs/sad_holiday_srm2832r9q?wsid=/subscriptions/56539498-d3d8-4a3b-92f4-f3b098a11d1e/resourcegroups/continuous_review_ms_and_ucl/workspaces/eppi_dev



In [21]:
from azure.ai.ml import Input

classify_unlabelled_tfidf = load_component(
    source="./final_components/classify_unlabelled_tfidf/classify_unlabelled_tfidf.yml"
)

unlabelled_tfidf_scores = Input(
    path=first_pipeline.outputs.unlabelled_tfidf_scores.path, type="uri_folder"
)
trained_model = Input(path=first_pipeline.outputs.trained_model.path, type="uri_folder")


@pipeline(default_compute=cluster_name, name="classify_unlabelled_tfidf_pipeline")
def classify_unlabelled_tfidf_pipeline(
    unlabelled_tfidf_scores,
    threshold,
    trained_model,
):
    classification_run = classify_unlabelled_tfidf(
        unlabelled_tfidf_scores=unlabelled_tfidf_scores,
        threshold=threshold,
        trained_model=trained_model,
    )

    return classification_run.outputs


third_pipeline = classify_unlabelled_tfidf_pipeline(
    unlabelled_tfidf_scores=unlabelled_tfidf_scores,
    threshold=threshold,
    trained_model=trained_model,
)

third_pipeline_job = ml_client.jobs.create_or_update(
    third_pipeline, experiment_name="classify_unlabelled_tfidf"
)

import datetime

print(f"current time: {datetime.datetime.now()}")

Uploading classify_unlabelled_tfidf.py (< 1 MB): 100%|██████████| 2.54k/2.54k [00:00<00:00, 49.6kB/s]


pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored


current time: 2025-03-13 15:26:28.774984
